# Data Importing and Cleaning

To avoid overcomplication and better readability, we have split the project into several sections including:
    - Background Introduction & Data Cleaning
    - EDA & Data Visualization
    - Regression Modelling, Model Evaluation & Conclusion

In [1]:
import pandas as pd
import numpy as np
from functions import *
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('data/analytic_data2019.csv', header = [0])
df.shape

(3195, 534)

In [3]:
df.columns = df.columns.str.replace(' ', '_')
df.drop([0,1], axis  = 0, inplace = True)

In [4]:
clean_columns = [x for x in df.columns if 
                 ("numerator" not in x) and 
                 ("denominator" not in x) and
                 ("CI_low" not in x) and
                 ("CI_high" not in x) and
                 ("Ratio" not in x) and
                 ("rate" not in x) and
                 ("_(White)" not in x) and
                 ("_(Black)" not in x) and
                 ("_(Hispanic)" not in x)]

df_clean = df[clean_columns]
df_clean = df_clean.drop(['State_FIPS_Code', 'County_FIPS_Code', '5-digit_FIPS_Code'], axis = 1)
df_clean.head()

,State_Abbreviation,Name,Release_Year,County_Ranked_(Yes=1/No=0),Premature_death_raw_value,Poor_or_fair_health_raw_value,Poor_physical_health_days_raw_value,Poor_mental_health_days_raw_value,Low_birthweight_raw_value,Adult_smoking_raw_value,Adult_obesity_raw_value,Food_environment_index_raw_value,Physical_inactivity_raw_value,Access_to_exercise_opportunities_raw_value,Excessive_drinking_raw_value,Alcohol-impaired_driving_deaths_raw_value,Sexually_transmitted_infections_raw_value,Teen_births_raw_value,Uninsured_raw_value,Primary_care_physicians_raw_value,Dentists_raw_value,Mental_health_providers_raw_value,Preventable_hospital_stays_raw_value,Mammography_screening_raw_value,Flu_vaccinations_raw_value,High_school_graduation_raw_value,Some_college_raw_value,Unemployment_raw_value,Children_in_poverty_raw_value,Income_inequality_raw_value,Children_in_single-parent_households_raw_value,Social_associations_raw_value,Violent_crime_raw_value,Injury_deaths_raw_value,Air_pollution_-_particulate_matter_raw_value,Drinking_water_violations_raw_value,Severe_housing_problems_raw_value,Percentage_of_households_with_high_housing_costs,Percentage_of_households_with_overcrowding,Percentage_of_households_with_lack_of_kitchen_or_plumbing_facilities,Driving_alone_to_work_raw_value,Long_commute_-_driving_alone_raw_value,Life_expectancy_raw_value,Premature_age-adjusted_mortality_raw_value,Child_mortality_raw_value,Infant_mortality_raw_value,Frequent_physical_distress_raw_value,Frequent_mental_distress_raw_value,Diabetes_prevalence_raw_value,HIV_prevalence_raw_value,Food_insecurity_raw_value,Limited_access_to_healthy_foods_raw_value,Drug_overdose_deaths_raw_value,Motor_vehicle_crash_deaths_raw_value,Insufficient_sleep_raw_value,Uninsured_adults_raw_value,Uninsured_children_raw_value,Other_primary_care_providers_raw_value,Disconnected_youth_raw_value,Median_household_income_raw_value,Children_eligible_for_free_or_reduced_price_lunch_raw_value,Residential_segregation_-_Black/White_raw_value,Residential_segregation_-_non-white/white_raw_value,Homicides_raw_value,Firearm_fatalities_raw_value,Homeownership_raw_value,Severe_housing_cost_burden_raw_value,Population_raw_value,%_below_18_years_of_age_raw_value,%_65_and_older_raw_value,%_Non-Hispanic_African_American_raw_value,%_American_Indian_and_Alaskan_Native_raw_value,%_Asian_raw_value,%_Native_Hawaiian/Other_Pacific_Islander_raw_value,%_Hispanic_raw_value,%_Non-Hispanic_white_raw_value,%_not_proficient_in_English_raw_value,%_Females_raw_value,%_Rural_raw_value,Communicable_disease_raw_value,Self-inflicted_injury_hospitalizations_raw_value,Cancer_incidence_raw_value,Coronary_heart_disease_hospitalizations_raw_value,Cerebrovascular_disease_hospitalizations_raw_value,Smoking_during_pregnancy_raw_value,Drug_arrests_raw_value,Opioid_hospital_visits_raw_value,Alcohol-related_hospitalizations_raw_value,On-road_motor_vehicle_crash-related_ER_visits_raw_value,Off-road_motor_vehicle_crash-related_ER_visits_raw_value,Childhood_immunizations_raw_value,Reading_proficiency_raw_value,W-2_enrollment_raw_value,Poverty_raw_value,Older_adults_living_alone_raw_value,Hate_crimes_raw_value,Child_abuse_raw_value,Injury_hospitalizations_raw_value,Fall_fatalities_65+_raw_value,Year_structure_built_raw_value,Male_population_0-17_raw_value,Male_population_18-44_raw_value,Male_population_45-64_raw_value,Male_population_65+_raw_value,Total_male_population_raw_value,Female_population_0-17_raw_value,Female_population_18-44_raw_value,Female_population_45-64_raw_value,Female_population_65+_raw_value,Total_female_population_raw_value,Population_growth_raw_value
2,AL,Alabama,2019,NaN,9917.2328984,0.2140240566,4.4004575204,4.577366748,0.1014549259,0.215381544,0.351,5.8,0.282,0.6164961831,0.1417814586,0.2859025033,543.6,33.113479936,0.107717996,0.000653877,0.0004761273,0.0009050726,5496,0.42,0.42,0.893,0.5986474992,0.0439678517,0.244,5.238579235,0.3810461622,12.242715851,479.91918191,79.908979052,11,0.0746268657,0.1491215019,NaN,NaN,NaN,0.858

In [5]:
for i in df_clean.columns[2:]:
    df_clean[i] = df_clean[i].astype('float64')

In [6]:
drop_na_columns(df_clean, df_clean.columns, threshold = 0.5)

Number of dropped columns: 35


Dropped columns: 
 ['Infant_mortality_raw_value', 'Disconnected_youth_raw_value', 'Homicides_raw_value', 'Communicable_disease_raw_value', 'Self-inflicted_injury_hospitalizations_raw_value', 'Cancer_incidence_raw_value', 'Coronary_heart_disease_hospitalizations_raw_value', 'Cerebrovascular_disease_hospitalizations_raw_value', 'Smoking_during_pregnancy_raw_value', 'Drug_arrests_raw_value', 'Opioid_hospital_visits_raw_value', 'Alcohol-related_hospitalizations_raw_value', 'On-road_motor_vehicle_crash-related_ER_visits_raw_value', 'Off-road_motor_vehicle_crash-related_ER_visits_raw_value', 'Childhood_immunizations_raw_value', 'Reading_proficiency_raw_value', 'W-2_enrollment_raw_value', 'Poverty_raw_value', 'Older_adults_living_alone_raw_value', 'Hate_crimes_raw_value', 'Child_abuse_raw_value', 'Injury_hospitalizations_raw_value', 'Fall_fatalities_65+_raw_value', 'Year_structure_built_raw_value', 'Male_population_0-17_raw_value', 'Male_population_18-44_raw_va

Remove total state rows in the county column

In [7]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 
          'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 
          'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 
          'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 
          'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 
          'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

to_drop_duplicates = list(df_clean[df_clean.Name.isin(states)].index)
dc_duplicate = [df_clean[df_clean.State_Abbreviation == 'DC'].index[0]]

df_clean.drop(to_drop_duplicates + dc_duplicate, axis = 0, inplace = True)

In [8]:
continuous_columns = df_clean.select_dtypes('float64')
continuous_na_filler(df_clean, continuous_columns, 'median')

In [9]:
renamed_columns = []

for i in df_clean.columns:
    if 'raw_value' in i:
        renamed_columns.append(i.replace('_raw_value', ''))
    else:
        renamed_columns.append(i)

In [10]:
df_clean.columns = renamed_columns

In [11]:
df_clean.head()

,State_Abbreviation,Name,Release_Year,County_Ranked_(Yes=1/No=0),Premature_death,Poor_or_fair_health,Poor_physical_health_days,Poor_mental_health_days,Low_birthweight,Adult_smoking,Adult_obesity,Food_environment_index,Physical_inactivity,Access_to_exercise_opportunities,Excessive_drinking,Alcohol-impaired_driving_deaths,Sexually_transmitted_infections,Teen_births,Uninsured,Primary_care_physicians,Dentists,Mental_health_providers,Preventable_hospital_stays,Mammography_screening,Flu_vaccinations,High_school_graduation,Some_college,Unemployment,Children_in_poverty,Income_inequality,Children_in_single-parent_households,Social_associations,Violent_crime,Injury_deaths,Air_pollution_-_particulate_matter,Drinking_water_violations,Severe_housing_problems,Percentage_of_households_with_high_housing_costs,Percentage_of_households_with_overcrowding,Percentage_of_households_with_lack_of_kitchen_or_plumbing_facilities,Driving_alone_to_work,Long_commute_-_driving_alone,Life_expectancy,Premature_age-adjusted_mortality,Child_mortality,Frequent_physical_distress,Frequent_mental_distress,Diabetes_prevalence,HIV_prevalence,Food_insecurity,Limited_access_to_healthy_foods,Drug_overdose_deaths,Motor_vehicle_crash_deaths,Insufficient_sleep,Uninsured_adults,Uninsured_children,Other_primary_care_providers,Median_household_income,Children_eligible_for_free_or_reduced_price_lunch,Residential_segregation_-_Black/White,Residential_segregation_-_non-white/white,Firearm_fatalities,Homeownership,Severe_housing_cost_burden,Population,%_below_18_years_of_age,%_65_and_older,%_Non-Hispanic_African_American,%_American_Indian_and_Alaskan_Native,%_Asian,%_Native_Hawaiian/Other_Pacific_Islander,%_Hispanic,%_Non-Hispanic_white,%_not_proficient_in_English,%_Females,%_Rural
3,AL,Autauga County,2019.0,1.0,8824.057123,0.184111,4.200578,4.306739,0.084757,0.191247,0.375,7.2,0.311,0.686775,0.168780,0.288136,341.2,26.524208,0.085010,0.000415,0.000324,0.000162,6599.0,0.44,0.41,0.900000,0.608547,0.038635,0.193,4.636664,0.247001,12.631731,272.282220,74.031808,11.7,0.0,0.149546,0.132052,0.024516,0.006374,0.859651,0.383,76.330589,438.7,53.279441,0.128244,0.126384,0.142,225.5,0.134,0.119911,9.623076,20.377158,0.359054,0.109611,0.024762,0.000306,58343.0,0.479317,29.945393,27.147569,17.7,0.732877,0.126452,55504.0,0.239370,0.151196,0.192545,0.004756,0.012792,0.001045,0.028575,0.744739,0.008285,0.513422,0.420022
4,AL,Baldwin County,2019.0,1.0,7224.632160,0.180605,4.098748,4.249649,0.083387,0.167955,0.310,8.0,0.238,0.719710,0.167150,0.318750,338.8,29.594009,0.106993,0.000729,0.000499,0.000912,3833.0,0.45,0.45,0.863616,0.661339,0.039883,0.147,4.518560,0.251841,10.692213,203.660396,69.374289,10.3,1.0,0.138317,0.125693,0.010728,0.006072,0.847194,0.405,78.599498,348.4,47.000017,0.126220,0.128226,0.113,163.9,0.123,0.054244,16.322612,15.092364,0.333059,0.135703,0.032380,0.000522,56607.0,0.450791,43.908361,33.240059,14.4,0.728593,0.127959,212628.0,0.218485,0.199470,0.089532,0.007760,0.011565,0.000687,0.045502,0.830474,0.004545,0.514528,0.422791
5,AL,Barbour County,2019.0,1.0,9586.165037,0.257734,5.067438,4.634994,0.109526,0.215409,0.443,5.6,0.282,0.536257,0.126987,0.296296,557.9,45.092838,0.125132,0.000385,0.000356,0.000079,4736.0,0.46,0.37,0.814103,0.367740,0.059009,0.503,5.755063,0.572157,8.472944,414.277861,72.900134,11.5,0.0,0.154555,0.136743,0.020065,0.008134,0.834044,0.338,75.779457,469.8,77.080027,0.162162,0.146323,0.180,436.0,0.232,0.107397,19.000000,20.966953,0.385632,0.165422,0.029014,0.000237,32490.0,0.737698,27.102488,24.313765,15.2,0.625068,0.135868,25270.0,0.207638,0.188247,0.479422,0.006529,0.004630,0.001860,0.042066,0.459557,0.011993,0.472299,0.677896
6,AL,Bibb County,2019.0,1.0,11783.543675,0.199969,4.363377,4.315710,0.111050,0.199164,0.378,7.6,0.349,0.162514,0.159252,0.272727,302.1,45.454545,0.096801,0.000574,0.000265,0.000088,5998.0,0.44,0.39,0.837638,0.484063,0.043851,0.273,4.254594,0.295654,10.157665,89.349126,100.077937,11.2,0.0,0.109609,0.108081,0.001993,0.002847,0.863659

Also, to avoid biases against any races, let's remove this so that no racial discimination will occur

In [12]:
race_columns = ['%_below_18_years_of_age', '%_65_and_older', '%_Non-Hispanic_African_American',
       '%_American_Indian_and_Alaskan_Native', '%_Asian',
       '%_Native_Hawaiian/Other_Pacific_Islander', '%_Hispanic',
       '%_Non-Hispanic_white', 'Residential_segregation_-_Black/White',
       'Residential_segregation_-_non-white/white']

df_clean.drop(race_columns, axis = 1, inplace = True)

In [13]:
df_clean.to_csv('clean_data.csv')